In [1]:
import asyncio 
import os
from enum import Enum 
from dotenv import load_dotenv
from semantic_kernel import Kernel
from semantic_kernel.kernel_pydantic import KernelBaseSettings
from semantic_kernel.agents import Agent, ChatCompletionAgent , ChatCompletionAgent , GroupChatOrchestration
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents.orchestration.group_chat import BooleanResult, GroupChatManager, MessageResult, StringResult
from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin, MCPStdioPlugin
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.contents import ChatHistory, ChatMessageContent, AuthorRole
from semantic_kernel.functions import KernelArguments
from semantic_kernel.connectors.ai.prompt_execution_settings import PromptExecutionSettings
from semantic_kernel.prompt_template import KernelPromptTemplate, PromptTemplateConfig
from semantic_kernel.agents.orchestration.group_chat import BooleanResult, StringResult, MessageResult
from typing import Callable, override, ClassVar
import re

In [2]:
kernel = Kernel()

class Service(Enum):
    OpenAI = "openai"
    AzureOpenAI = "azureopenai"
    HuggingFace = "huggingface"

class ServiceSettings(KernelBaseSettings):
    global_llm_service: str | None = None
    
load_dotenv(override=True)

True

In [3]:
service_settings = ServiceSettings()

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = (
    Service.AzureOpenAI
    if service_settings.global_llm_service is None
    else Service(service_settings.global_llm_service.lower())
)
print(f"Using service type: {selectedService}")

Using service type: Service.AzureOpenAI


In [4]:
# Remove all services so that this cell can be re-run without restarting the kernel
kernel.remove_all_services()


# kernel에 등록한 서비스는 -> agent를 만들때 자동으로 쓰이진않음
#나중에 쓰려고 등록하는것. 

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(
            service_id=service_id,
        ),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
        ),
    )

In [5]:
web_search_plugin  = MCPStreamableHttpPlugin(name="web_search_mcp", description= "Retrieve websearch information using duckduck go ",url="http://127.0.0.1:8000/mcp") 
microsoft_learn_search_plugin = MCPStreamableHttpPlugin(name="microsoft_learn_mcp", url=str(os.getenv("MICROSOFT_LEARN_MCP_URL")))


## MultiAgent Architecture
### 1. Sequential Orchestration
- 정해진 순서에 따라 순차처리, 에이전트의 OUTPUT을 뒤쪽 에이전트에 전달 
- Ideal: 전 에이전트의 처리결과를 바탕으로 현재의 에이전트가 동작하는 케이스에 적합
- EX) Document Review, Data Processing Pipelines, MuiltiStage Reasoning

### 2. Concurrent Orchestration
- 멀티 에이전트들이 병렬적으로 같은 태스크를 수행하는 구조. 
- Input을 독립적으로 처리하고, Result는 Collect되고 aggregated됨
- Ideal: 다양한 시각으로 생각해야하는 태스크의 적합함
- EX) Brainstorming, Ensemble Reasoning Voting System


### 3. GroupChat Orchestration
- 모든 Agent와 그리고 Human이 하나의 GroupChat안에서 Chat Manager중심으로 협업함. 
- Manager가 응답순서를 조정하고, 필요시 인간 개입
- 에이전트간 대화기반 협업하며 매니저가 대화흐름을 조율함 + 인간이 Converstaion 참여도 가능 
- 매니저를 거쳐서 Complete을 판단하고 Result를 낸다. 
- Ideal: 다양한 기능을 가진 에이전트가 유연하게 협력적 역할을 할떄
- EX) Meeting, Debates or collaborative problem-solving sessions


### 4. Handoff Orchestration
- 같은 레벨의 에이전트들이, Context와 User Input에 기반해 컨트롤을 서로 넘겨가며 협업
- 매니저는 존재하지않지만, 유저 Input을 받아야하기에 하나의 General한 에이전트는 필요
- GroupChat과의 차이는 모든 Agent에서 바로 Complete를 하고 Result를 리턴할 수 있다는 것.
- Ideal: 다이나믹하게 Task를 위임해야하는 케이스에 유리. 타 에이전트에 태스크를 넘겨줄때 매니저를 거치지 않기떄문에 오버헤드 더 적음
- Ex) Customer Support, Expert System, any scnario requiring dynamic delgation 


### 5. Magnetic Orchestration
- 에이전트 Orchestrator와 Specialized Agents들이 Task Ledger와 Process Ledger를 통해,  플래닝을 하고 프로세스를 기록하며 TASK 진행. 총 4가지 개념 이해 필요 
- Task Ledger: Orchestrator가 태스크를 처음 받아, Planning과 기타 요소들을 기록하는 곳 (Given or Verified Facts, Facts to llok up, Facts to derive, Educated Guesses, Task Plan) 
- Process Ledger: 에이전트들이 태스크 수행 프로세스를 기록하는 곳 (Task Complete, Unproductive Loops, Is progress being made, what is the next speaker, Next speaker instruction)
- Inner Loop(Progress Ledger Update Loop): Progress가 잘 진행되고있는 판별하는 루프임 Task Complete? -> Progress Made? -> Stall Count >2 -> Progress LEdger UPdatee or agent call 
- Outer Loop(Progress Ledger Update Loop): Task를 plan하고 조정하는 루프임. Count 2번안에 진전이 없으면 Task Ledger업데이트 통해 새롭게 플랜 

In [6]:
from typing import List, Tuple
from semantic_kernel import Kernel

async def create_kernel_azurechatcompletion(service_id: str, plugins: List[Tuple]) -> Kernel:
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id=service_id))
    
    for plugin_instance, plugin_name in plugins:
        await plugin_instance.connect()
        print("connect Successful")
        kernel.add_plugin(plugin_instance, plugin_name=plugin_name)
    
    return kernel

async def get_agents() -> list[Agent]:
    kernel_web = await create_kernel_azurechatcompletion("web_search_agent", [(web_search_plugin, "web_search_plugin")])
    web_search_agent = ChatCompletionAgent(
        name="Web_Search_Agent",
        description="Web search agent grounding web information in DuckDuckGo",
        instructions=(
            """
            You are a Web Search Agent supporting Microsoft tech knowledge queries.

            Your role is to search for supplemental or external information that is not available on Microsoft Learn.  
            You must use the DuckDuckGo MCP tool whenever the user's question requires additional context, recent updates, or general web information beyond official Microsoft documentation.

            Never rely on your own knowledge.  
            You must always call the DuckDuckGo tool and include its result in your response.  
            Do not omit any key detail or fact from the tool response.  
            Answer clearly and concisely based only on retrieved information.

            """
        ),
        kernel=kernel_web,
    )

    kernel_microsoft_learn = await create_kernel_azurechatcompletion("microsoft_learn_mcp", [(microsoft_learn_search_plugin, "microsoft_learn_search_plugin")])
    microsoft_learn_search_agent = ChatCompletionAgent(
        name="Microsoft_Learn_Search_Agent",
        description="Museum Search agent grounding Met Museum info in internal DB",
        instructions=(
            """
            You are a Microsoft Learn Knowledge Agent.  
            You specialize in retrieving official Microsoft documentation and training content using the Microsoft Learn MCP tool.

            When the user asks about Microsoft products, services, certifications, tools, or training,  
            you must call the Microsoft Learn MCP tool to retrieve official guidance.

            Your answer must be fully based on the retrieved result, not on assumptions.  
            Never omit important details or steps from the response.  
            If any section is unclear or incomplete, request support from the Web_Search_Agent.
            Be authoritative, concise, and accurate.
            
            """
        ),
        kernel=kernel_microsoft_learn
    )

    return [web_search_agent, microsoft_learn_search_agent]


In [7]:
# import asyncio
# from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
# import nest_asyncio

# async def main():
#     agent_list = await get_agents()  # ✅ await 사용
#     web_search_agent = agent_list[0]
#     print(web_search_agent.kernel.plugins)

#     USER_INPUTS = [
#         "Please search the web and tell me when the Metropolitan Museum of Art was founded.",
#     ]

#     thread: ChatHistoryAgentThread = None

#     for user_input in USER_INPUTS:
#         print(f"# User: '{user_input}'")
#         first_chunk = True
#         async for response in web_search_agent.invoke_stream(messages=user_input, thread=thread):
#             thread = response.thread
#             if first_chunk:
#                 print(f"# {response.name}: ", end="", flush=True)
#                 first_chunk = False
#             print(response.content, end="", flush=True)
#         print()

#     if thread:
#         await thread.delete()

# nest_asyncio.apply()
# await main()


In [ ]:

class MuseumSmartManager(GroupChatManager):
    topic:str
    service: ChatCompletionClientBase
    """
    Custom GroupChatManager that delegates query to WebSearchAgent or MuseumSearchAgent
    based on user intent and summarizes when enough context is available.
    """


    selection_prompt: ClassVar[str] = (
    "You are a smart discussion manager helping a user query about Microsoft technologies.\n"
    "You are given the user’s question and previously retrieved information below.\n"
    "Decide which agent should handle the query: 'Web_Search_Agent' or 'Microsoft_Learn_Search_Agent'.\n\n"
    "Rules:\n"
    "- If the user is asking for **official Microsoft product documentation, tutorials, certification guidance, or feature-specific usage**, choose 'Microsoft_Learn_Search_Agent'.\n"
    "- If the user is asking for **community opinions, comparisons, news, announcements, blog content, GitHub info, or anything not formally in Microsoft Learn**, choose 'Web_Search_Agent'.\n"
    "- If the question is vague or it’s unclear where the info is best found, default to 'Web_Search_Agent'.\n\n"
    "Respond with the agent name only.\n\n"
    "Previous web retrieved information {{$web_retrieved}}"
    "Previous microsoft-learned retrieved information {{$microsoft_learn_retrieved}}"
    "User query: {{$query}}"
    )

    
    termination_prompt: ClassVar[str] = (
    "You are a mediator checking whether the user's question about Microsoft technology has been fully answered.\n"
    "Only answer True if ALL of the following conditions are met:\n"
    " • At least one agent (Web_Search_Agent or Microsoft_Learn_Search_Agent) provided a clear and relevant answer;\n"
    " • The user's intent is fully addressed with accurate and useful information;\n"
    " • If the user implicitly or explicitly expects answers from both General Web and Microsfot-Learn sources, then responses from BOTH AGENT must be present;\n"
    " • No further clarification or follow-up seems necessary.\n\n"
    "Otherwise, respond with False.\n"
    "Respond with exactly True or False."
    )



    summarize_prompt: ClassVar[str] = (
        "You are generating a comprehensive and faithful response based on a multi-agent discussion about a Microsoft technology question.\n"
        "Your goal is not to shorten, but to carefully integrate all information provided by the different agents.\n\n"
        "Structure your response in three clearly separated parts:\n\n"
        "1. **Restating Your Query**\n"
        "   - Briefly rephrase or repeat what the user originally asked.\n\n"
        "2. **Source-Based Information (Verbatim + Merged)**\n"
        "   - Present findings from each source without omitting important content.\n"
        "   - Do not summarize or paraphrase too aggressively.\n"
        "   - Use this exact format:\n"
        "     [Source: Web]\n"
        "     {Consolidated but accurate representation of everything provided by the Web_Search_Agent.}\n\n"
        "     [Source: MS Learn]\n"
        "     {Consolidated but accurate representation of everything provided by the MS_Learn_Search_Agent.}\n"
        "   - If only one source was used, include just that one.\n\n"
        "3. **Final Answer**\n"
        "   - Integrate the above into a single, clear answer that fully covers the user's question.\n"
        "   - Do not introduce any new information not already included above.\n"
        "   - Ensure nothing important is lost in the integration.\n\n"
        "Output must preserve key details, and must not reduce or skip over information. Accuracy and completeness are more important than brevity."
    )



    def __init__(self, topic: str, service, **kwargs) -> None:
        super().__init__(topic=topic, service=service, **kwargs)

    async def _render_prompt(self, prompt: str, arguments: KernelArguments) -> str:
        """프롬프트를 입력 인자와 함께 렌더링해주는 헬퍼 함수"""
        prompt_template = KernelPromptTemplate(prompt_template_config=PromptTemplateConfig(template=prompt))
        return await prompt_template.render(Kernel(), arguments=arguments)

    @override
    async def should_request_user_input(self, chat_history: ChatHistory) -> BooleanResult:
        """항상 False 반환 -> 사용자 개입 불필요"""
        return BooleanResult(result=False, reason="User input not required between rounds.")

    @override
    async def select_next_agent(self, chat_history: ChatHistory, participant_descriptions: dict[str, str]) -> StringResult:
        """다음으로 응답할 에이전트 선택"""
        
        #1. Find User Query: ChatHistory에서 마지막 USER 메시지를 찾음 (최신 쿼리)
        user_message = next((m for m in reversed(chat_history.messages) if m.role == AuthorRole.USER), None)
            
        # 2. 모든 Web_Search_Agent와 Microsoft_Learn_Search_Agent의 응답 집계
        web_retrieved_list = []
        microsoft_learn_retrieved_list = []

        for msg in chat_history.messages:
            if msg.role == AuthorRole.ASSISTANT:
                if msg.name == "Web_Search_Agent":
                    web_retrieved_list.append(msg.content.strip())
                elif msg.name == "Microsoft_Learn_Search_Agent":
                    microsoft_learn_retrieved_list.append(msg.content.strip())

        web_retrieved = "\n---\n".join(web_retrieved_list) if web_retrieved_list else "[None]"
        microsoft_learn_retrieved = "\n---\n".join(microsoft_learn_retrieved_list) if microsoft_learn_retrieved_list else "[None]"

        # 3. 프롬프트 렌더링
        rendered_prompt = await self._render_prompt(
            self.selection_prompt,
            KernelArguments(
                query=user_message,
                web_retrieved=web_retrieved,
                microsoft_learn_retrieved=microsoft_learn_retrieved,
            )
        )

        # 프롬프트로 만든 System 메시지를 가장 앞에 삽입 (최우선 컨텍스트)
        chat_history.messages.insert(0, ChatMessageContent(role=AuthorRole.SYSTEM, content=rendered_prompt))
        chat_history.add_message(ChatMessageContent(role=AuthorRole.USER, content="Choose the best agent."))

        response = await self.service.get_chat_message_content(chat_history, settings=PromptExecutionSettings(response_format=StringResult))
        
        try:
            result = StringResult.model_validate_json(response.content)

        except Exception as e: 
            print("Failed to parse agent selection response:", response.content)
            raise e

        print("=== [AGENT SELECTION] ===")
        print(f"Selected agent: {result.result}")
        print(f"Reason: {result.reason}\n")

        if result.result not in participant_descriptions:
            raise ValueError(f"Invalid agent selected: {result.result}")

        return result

    @override
    async def should_terminate(self, chat_history: ChatHistory) -> BooleanResult:
        """그룹챗 종료 여부 판단"""
            # 1. Web/Museum agent가 실질적으로 응답했는지 확인
        agent_names = {"Web_Search_Agent", "Microsoft_Learn_Search_Agent"}
        
        retrieval_related_agent_messages = [
            m for m in chat_history.messages
            if m.role == AuthorRole.ASSISTANT and m.name in agent_names and m.content.strip()
        ]

        if len(retrieval_related_agent_messages) < 1:
            return BooleanResult(
                result=False,
                reason="No valid responses from Web or Museum agent yet."
            )
            
        
        rendered_prompt = await self._render_prompt(self.termination_prompt, KernelArguments(topic=self.topic))

        chat_history.messages.insert(0, ChatMessageContent(role=AuthorRole.SYSTEM, content=rendered_prompt))
        chat_history.add_message(ChatMessageContent(role=AuthorRole.USER, content="Should we summarize now?"))

               
        
        response = await self.service.get_chat_message_content(chat_history, settings=PromptExecutionSettings(response_format=BooleanResult))
        result = BooleanResult.model_validate_json(response.content)

        print("=== [TERMINATION DECISION] ===")
        print(f"Terminate: {result.result}")
        print(f"Reason: {result.reason}\n")

        return result

    @override
    async def filter_results(self, chat_history: ChatHistory) -> MessageResult:
        """최종 요약 결과 반환"""
        rendered_prompt = await self._render_prompt(self.summarize_prompt, KernelArguments(topic=self.topic))

        chat_history.messages.insert(0, ChatMessageContent(role=AuthorRole.SYSTEM, content=rendered_prompt))
        chat_history.add_message(ChatMessageContent(role=AuthorRole.USER, content="Generate a comprehensive answer based on all above."))

        response = await self.service.get_chat_message_content(chat_history, settings=PromptExecutionSettings(response_format=StringResult))
        summary = StringResult.model_validate_json(response.content)

        print("=== [Answer RESULT] ===")
        print(f"Answer: {summary.result}")
        print(f"Reason: {summary.reason}\n")

        return MessageResult(
            result=ChatMessageContent(role=AuthorRole.ASSISTANT, content=summary.result),
            reason=summary.reason,
        )

In [23]:
def agent_response_callback(message: ChatMessageContent) -> None:
    """Callback function to retrieve agent responses."""
    print(f"**{message.name}**\n{message.content}")


async def main():
    """Main function to run the agents."""
    # 1. Create a group chat orchestration with the custom group chat manager
    agents = await get_agents()
    group_chat_orchestration = GroupChatOrchestration(
        members=agents,
        manager=MuseumSmartManager(
            topic="Microsoft Technologies",
            service=AzureChatCompletion(),
            max_rounds=10,
        ),
        agent_response_callback=agent_response_callback,
    )

    # 2. Create a runtime and start it
    runtime = InProcessRuntime()
    runtime.start()

    # 3. Invoke the orchestration with a task and the runtime
    orchestration_result = await group_chat_orchestration.invoke(
        task="how can i use copilot studio find in both web and microsoft-learn site",
        runtime=runtime,
    )

    # 4. Wait for the results
    value = await orchestration_result.get()
    print(value)

    # 5. Stop the runtime after the invocation is complete
    await runtime.stop_when_idle()


In [24]:
import nest_asyncio

nest_asyncio.apply()
if __name__ == "__main__":
    asyncio.run(main())

connect Successful
connect Successful
=== [AGENT SELECTION] ===
Selected agent: Microsoft_Learn_Search_Agent
Reason: The user is asking for official guidance on how to use Copilot Studio, which is a Microsoft product feature. This type of query is best served by Microsoft Learn for official documentation, tutorials, and feature usage instructions.

**Microsoft_Learn_Search_Agent**
You can use Microsoft Copilot Studio to build and find custom AI copilots both on the web and within Microsoft Learn resources as follows:

1. **Using Copilot Studio on the Web:**
   - Access Copilot Studio via the web app at https://copilotstudio.microsoft.com.
   - This platform enables IT admins and advanced users to create AI agents for tasks or customer interactions.
   - The web app supports advanced agent concepts such as entities and variables, allowing creation of complex agents.
   - You can try the Copilot Studio demo here: https://copilotstudio.microsoft.com/tryit?azure-portal=true.

2. **Using Co